In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName("tipapp").getOrCreate()

In [7]:
df=spark.read.csv("f:\\datasets\\tips.csv",header=True,inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [8]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [11]:
from pyspark.ml.feature import VectorAssembler

In [26]:

from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip'],outputCol='tipData')
dff=featureassembler.transform(df)
dff.show()


+----------+----+------+------+---+------+----+-------+
|total_bill| tip|   sex|smoker|day|  time|size|tipData|
+----------+----+------+------+---+------+----+-------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2| [1.01]|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3| [1.66]|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|  [3.5]|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| [3.31]|
|     24.59|3.61|Female|    No|Sun|Dinner|   4| [3.61]|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| [4.71]|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|  [2.0]|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4| [3.12]|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2| [1.96]|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2| [3.23]|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| [1.71]|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|  [5.0]|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2| [1.57]|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|  [3.0]|
|     14.83|3.02|Female|    No|Sun|Dinner|   2| 

In [27]:
dff.select('tipData','total_bill').show()

+-------+----------+
|tipData|total_bill|
+-------+----------+
| [1.01]|     16.99|
| [1.66]|     10.34|
|  [3.5]|     21.01|
| [3.31]|     23.68|
| [3.61]|     24.59|
| [4.71]|     25.29|
|  [2.0]|      8.77|
| [3.12]|     26.88|
| [1.96]|     15.04|
| [3.23]|     14.78|
| [1.71]|     10.27|
|  [5.0]|     35.26|
| [1.57]|     15.42|
|  [3.0]|     18.43|
| [3.02]|     14.83|
| [3.92]|     21.58|
| [1.67]|     10.33|
| [3.71]|     16.29|
|  [3.5]|     16.97|
| [3.35]|     20.65|
+-------+----------+
only showing top 20 rows



In [28]:
from pyspark.ml.regression import LinearRegression

In [30]:
train,test=dff.randomSplit([.70,.30])

In [32]:
lr=LinearRegression(featuresCol='tipData',labelCol='total_bill')
tm=lr.fit(train)
print(tm.intercept)
print(tm.coefficients)

6.670397179248463
[4.246280396665198]


In [33]:
result=tm.evaluate(test)

In [34]:
result.predictions.show()

+----------+----+------+------+----+------+----+-------+------------------+
|total_bill| tip|   sex|smoker| day|  time|size|tipData|        prediction|
+----------+----+------+------+----+------+----+-------+------------------+
|      7.51| 2.0|  Male|    No|Thur| Lunch|   2|  [2.0]|15.162957972578859|
|      8.51|1.25|Female|    No|Thur| Lunch|   2| [1.25]| 11.97824767507996|
|      8.77| 2.0|  Male|    No| Sun|Dinner|   2|  [2.0]|15.162957972578859|
|      9.55|1.45|  Male|    No| Sat|Dinner|   2| [1.45]|   12.827503754413|
|      9.94|1.56|  Male|    No| Sun|Dinner|   2| [1.56]| 13.29459459804617|
|     10.07|1.25|  Male|    No| Sat|Dinner|   2| [1.25]| 11.97824767507996|
|     10.29| 2.6|Female|    No| Sun|Dinner|   2|  [2.6]|17.710726210577977|
|     10.51|1.25|  Male|    No| Sat|Dinner|   2| [1.25]| 11.97824767507996|
|     10.65| 1.5|Female|    No|Thur| Lunch|   2|  [1.5]| 13.03981777424626|
|     12.02|1.97|  Male|    No| Sat|Dinner|   2| [1.97]|15.035569560678903|
|      12.6|

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [36]:
print('this is R2:',result.r2)
print('this is mean absolute error:',result.meanAbsoluteError)
print('Root mean square error:',result.rootMeanSquaredError)


this is R2: 0.4911096774990752
this is mean absolute error: 4.4168755181257175
Root mean square error: 6.057405951228092
